# Lindhardt susceptibility $\chi_0$ for a non interacting electron gas


The Lindhard function is defined by the equation

\begin{equation}
  \chi_0(\mathbf{q}, i\omega_n) = 
    -\frac{1}{\beta N}\sum_{\mathbf{k}, m} 
    G_0(\mathbf{k}, i\nu_m)G_0(\mathbf{k}+\mathbf{q}, i\omega_n + i\nu_m) 
\end{equation}

where $\mathbf{q}$ and $\mathbf{k}$ are momenta and $i\omega_n$ and $i\nu_m$ are Bosonic and Fermionic Matsubara frequencies, respectively.

Since the equation is a convolution in both momentum and frequency space the calculation can be done in fewer operations by Fourier transforming to imaginary time $\tau$ and real space $\mathbf{r}$ for the Green's function, using fast Fourier transforms (FFT) to compute

$$
G_0(\mathbf{r}, \tau) = 
  \mathcal{F}_{\{\mathbf{k}, i\nu_m\} \rightarrow \{\mathbf{r}, \tau\}} 
  \big\{ G_0(\mathbf{k}, i\omega_n) \big\}
$$

in real space $\mathbf{r}$ and imaginary time $\tau$ the convolution for $\chi_0$ is a simple product (since it corresponds to a simple bubble diagram with a particle-hole excitation)

$$
\chi_0(\mathbf{r},\tau) = G_0(\mathbf{r},\tau)G_0(-\mathbf{r},\beta -\tau)
$$

and finally one goes back to momentum and Matsubara frequency by two more fast Fourier transforms

$$ 
  \chi_0(i\omega_n, k) \equiv 
  \mathcal{F}_{\{\mathbf{r},\tau\} \rightarrow \{\mathbf{k}, i\omega_n\}} 
  \big\{ \chi_0(\mathbf{r}, \tau) \big\}
$$

Q: **How many operations are saved by using fast Fourier transforms rather than evaluating the convolutions?**

Q: **At what momenta $\mathbf{k}$ does the susceptibility $\chi_0$ have a maxima at zero frequency?**

Q: **How is the postiion of that maximum related to the Fermi surface?**

Q: **Can you plot the function at different temperatures and understand what happens physically?**
   

## Computing $\chi_0$ in C++

While it is possible to compute $\chi_0$ in Python, it is relatively slow, due to the required double loop over $r$ and $\tau$. 

Here we illustrate the C++ layer of the `TRIQS` library, using the `TRIQS/cpp2py` tool to wrap Python and C++
in a simple case.

The function bubble below computes the Lindhardt function performing the three steps described above.

The following cell compiles the function "bubble" in C++, exposes ("wraps") it to Python and loads it
in the current scope.

In [ ]:
%reload_ext cpp2py.magic

May take a few seconds to compile ...

In [ ]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

// The type of a Green function : (k,omega) -> Complex number
using g_k_w_type = gf_view<cartesian_product<brillouin_zone, imfreq>, scalar_valued>;
using g_r_t_type = gf<cartesian_product<cyclic_lattice, imtime>, scalar_valued>;

g_k_w_type bubble(g_k_w_type g0) {
    
    // Fourier Transformation of k, \omega to obtain g(r,t)
    auto grt = make_gf_from_fourier<0,1>(g0);
    
    // The mesh of gtr is a cartesian product mt x mr. We decompose it.
    auto [mr, mt] = grt.mesh();
    
    // The inverse temperature from the mesh
    double beta = mt.domain().beta;
    
    // A new mesh for chi, with a bosonic statistics, but same size as mt.
    auto mtb = gf_mesh<imtime>{beta, Boson, mt.size()};
    
    // Build chi (r, tau) with this new mesh.
    auto chi0 = g_r_t_type{{mr, mtb}};

    // we fill chi : chi(tau, r) = g(beta - tau, -r) * g(tau, r)
    for (auto const &r : mr)      
        for (auto const &t : mtb) 
            chi0[r, t] = grt(-r, beta - t) * grt(r, t); 

    // Fourier transform back to k, \omega space and return
    return make_gf_from_fourier<0,1>(chi0);
}

## Static Lindhardt susceptibility $\chi_0(k, \omega=0)$

The square lattice with nearest-neighbour hopping $t$ has a property called "perfect nesting", meaning that large parts of the fermi surface are wrapped on-to each other by a single momentum transfer $\mathbf{k}_{nesting}$, where $\mathbf{k}_{nesting}$ is called the "nesting vector". 

Go back to the fermi-surface polot of $\epsilon(\mathbf{k})$ and determine this peculiar momentum vector.

The "perfect nesting" greatly enhances the particle-hole susceptibility $\chi_0$ of the system and the Static Lindhardt susceptibility $\chi_0(\mathbf{k}, \omega=0)$ has a dominant peak at this momentum.

TODO : 
- Physics discussion ?

## Load single-particle Green's function $G_0(\mathbf{k}, i\omega_n)$

In [ ]:
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct, Idx
from pytriqs.archive import HDFArchive

with HDFArchive("tpsc.h5",'r') as R:
    g0 = R['g0_kw']

## Compute the Lindhardt susceptibility $\chi_0(\mathbf{k}, i\omega_n)$

Maybe ask to plot it vs Matsubara frequencies to see that at those temperatures it is essentially
non-zero only very close to 0.

In [ ]:
chi0_kw = bubble(g0)
print chi0_kw

### Structure in Matsubara frequency $i\omega_n$

Check that the Matsubara frequency window used $n_w$ is sufficient to capture the frequency dependence of $\chi_0(\mathbf{k}, i\omega_n)$ by plotting it for some momentum $\mathbf{k}$

In [ ]:
from pytriqs.plot.mpl_interface import oplot
oplot(chi0_kw[Idx(0, 0, 0), :])
oplot(chi0_kw[Idx(10, 0, 0), :])
oplot(chi0_kw[Idx(100, 0, 0), :])

## Save the bubble for later use

In [ ]:
from pytriqs.archive import HDFArchive
with HDFArchive("tpsc.h5") as R:
    R['chi0_kw'] = chi0_kw

## Plot the suscptibility

In [ ]:
%matplotlib inline
from pytriqs.plot.mpl_interface import plt
import numpy as np

nk = g0.mesh[0].linear_dims[0]+1
k = np.linspace(0, 2*np.pi, nk, endpoint=True)
kx, ky = np.meshgrid(k, k)

chi = lambda kx, ky: chi0_kw([kx,ky,0],0).real

# 2d plot
plt.figure(figsize=(3.25*2, 5))
plt.title('Static Lindhardt susceptibility $\chi_0(k, \omega=0)$')
plt.pcolor(kx, ky, np.vectorize(chi)(kx,ky))
plt.xlim(0, 2*np.pi)
plt.ylim(0, 2*np.pi)
plt.xticks([0, np.pi, 2*np.pi],[r"0",r"$\pi$",r"$2\pi$"])    
plt.yticks([0, np.pi, 2*np.pi],[r"0",r"$\pi$",r"$2\pi$"])
plt.colorbar()
plt.tight_layout()
plt.xlabel(r'$k_x$')
plt.ylabel(r'$k_y$')


# Here is a 3d plot. With "matplotlib inline" one cannot interact
from mpl_toolkits.mplot3d import axes3d, Axes3D
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(kx, ky, np.vectorize(chi)(kx,ky), cmap="jet")
ax.set_xlabel(r"$k_x$")
ax.set_ylabel(r"$k_y$")
ax.set_zlabel(r"$\chi^{0}$")

## Static susceptibility $\chi_0(\mathbf{k}, \omega=0)$ along high symmetry paths in the Brillouin zone

The static susceptibility along the high-symmetry path $\Gamma-X-M-\Gamma$ shows a dramatic enhancement at $\mathbf{k} = (\pi, \pi)$ due to the perfectly nested Fermi surface.

In [ ]:
from k_space_viz import k_space_path

G = np.array([0, 0, 0])
X = np.array([1, 1, 0]) * np.pi
M = np.array([1, 0, 0]) * np.pi

k_vecs, k_plot, K_plot = k_space_path([(G, X), (X, M), (M, G)])
kx, ky, kz = k_vecs.T

plt.figure(figsize=(6,4))
plt.title('Static Lindhardt susceptibility $\chi_0(k,\omega=0)$')
plt.plot(k_plot, np.vectorize(chi)(kx, ky))

plt.grid(); plt.axes().set_xticks(K_plot)
plt.xlim([K_plot.min(), K_plot.max()])
plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$']);
plt.ylabel(r'$\chi_0(k,\omega=0)$')
plt.tight_layout()
#plt.savefig('figure_chi0_k_bandpath.pdf')